In [ ]:
import csv
from collections import Counter, defaultdict
from tqdm import tqdm
import json
import numpy as np
import gzip
from datetime import datetime
from sentence_transformers import SentenceTransformer
import os
import re
import joblib
from sklearn import metrics
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# your modules are imported here
from indexing import Indexer, BasicInvertedIndex
from document_preprocessor import RegexTokenizer, Doc2QueryAugmenter
from ranker import Ranker, BM25, CrossEncoderScorer
from vector_ranker import VectorRanker
from l2r import L2RFeatureExtractor, L2RRanker
from vectordb_ranker import VectorDBRanker

In [ ]:
from importlib import reload
import indexing
reload(indexing)
from indexing import Indexer
import document_preprocessor
reload(document_preprocessor)
from document_preprocessor import RegexTokenizer, Doc2QueryAugmenter
import l2r
reload(l2r)
from l2r import L2RFeatureExtractor, L2RRanker

In [ ]:
data_prefix = '../data/'
model_prefix = '../models/'

In [ ]:
with open(data_prefix + 'rec_cats.json', 'r') as f:
    rec_cats = json.load(f)
    five_cats = rec_cats['short']
    all_cats = rec_cats['full']
with open(data_prefix + 'doc_cat_info.json', 'r') as f:
    doc_cat_info = json.load(f)
    doc_cat_info = {int(k):v for k, v in doc_cat_info.items()}

In [ ]:
doc_preproc = RegexTokenizer('\\w+')
stopwords = set()
with open(data_prefix + 'stopwords.txt', 'r', encoding='utf-8') as file:
    for stopword in file:
        stopwords.add(stopword.strip())
f'Stopwords collected {len(stopwords)}'

'Stopwords collected 543'

In [ ]:
doc_index = BasicInvertedIndex()
doc_index.load(data_prefix + 'doc_index')
tit_index = BasicInvertedIndex()
tit_index.load(data_prefix + 'title_index')

In [ ]:
with open(data_prefix + 'raw_text.json', 'r') as f:
    raw_text_dict = json.load(f)
    raw_text_dict = {int(k):v for k, v in raw_text_dict.items()}

In [ ]:
ce_scorer = CrossEncoderScorer(raw_text_dict)

In [ ]:
na_feat_extract = L2RFeatureExtractor(doc_index, tit_index, doc_cat_info, doc_preproc, stopword, set(all_cats), ce_scorer)

In [ ]:
na_model = joblib.load(model_prefix + 'na_model.joblib')

: 

In [ ]:
with open(data_prefix + 'test_data.json', 'r') as f:
    test_data = json.load(f)
    test_queries = list(test_data.keys())

In [ ]:
vdb_ranker = VectorDBRanker(raw_text_dict)

In [ ]:
bienc_nq_na_ranker = L2RRanker(doc_index, tit_index, doc_preproc, stopword, vdb_ranker, na_feat_extract)
bienc_nq_na_ranker.model.lgbmranker = na_model
bienc_nq_na_scores = [bienc_nq_na_ranker.query(query) for query in test_queries]